<a href="https://colab.research.google.com/github/it-ces/PUBLIC-AI/blob/main/OPTIMIZATION/Genetic_TSP(BIT).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
## Iván Andrés Trujillo Abella
## ivantrujillo1229@gmail.com

In [14]:
! pip install deap > null

In [15]:
# Genetic TSP implementation:
import random
import json
import numpy as np
import matplotlib.pyplot as plt
from deap import algorithms
from deap import creator
from deap import base
from deap import tools

In [16]:
import pandas as pd
URL = "https://raw.githubusercontent.com/it-ces/GA-OPTA-PUJ/main/cities.csv"
data = pd.read_csv(URL)
data = data[data['capital']=='admin'].sample(25, random_state=4).reset_index(drop=True)
capitals = data['city'].unique()
cities_dict = {}
for index in data.index:
    cities_dict[data.loc[index,'city']] = [data.loc[index, 'lat'] , data.loc[index, 'lng']]

In [17]:
# Fitness function and another neccesary funtcions
def distanceF(cityA, cityB):
    return ((cityA[0] - cityB[0])**2  + (cityA[1] - cityB[1])**2)**(1/2)

# Evaluate a path
def eval_tsp(individual):
    index_test = individual
    path_test = capitals[index_test]
    route = distanceF(cities_dict[path_test[0]],  cities_dict[path_test[-1]]) # The first and the last city to complete
    for i in range(len(path_test)-1):
        route += distanceF(cities_dict[path_test[i]], cities_dict[path_test[i+1]])
    return route, # It is important return a tuple


IND_SIZE = len(capitals)  # the number of cities to find the closest path

# Creating problem and kind of individual:
creator.create("FitnessMin", base.Fitness, weights = (-1.0,))
creator.create("Individual", list , fitness=creator.FitnessMin)
## Toolbox
## create the tours or paths..
#the strategy is store the list in a list and create indices as individuals or tours.
toolbox = base.Toolbox()
toolbox.register("indices", random.sample, range(IND_SIZE), IND_SIZE)   # draw k samples without replacement including
                                                                        # zero  and excluding N
#Given that "indices" returns us a complete individual dont need uses init.Repeat otherwise, init.Iterate
toolbox.register("individual", tools.initIterate, creator.Individual, toolbox.indices)
# two paratermers accep initIterate ( container(Individual) and Func(indices a iterable function))
# Create the initial population
toolbox.register("population", tools.initRepeat, list, toolbox.individual,100)

#Genetic Operators (preserve the names)
toolbox.register("mate", tools.cxOrdered)
toolbox.register("mutate", tools.mutShuffleIndexes, indpb=0.01)
toolbox.register("select", tools.selTournament, tournsize=3)
toolbox.register("evaluate", eval_tsp)

# The genetic algorithm
# MuPlusLambda
# This algorithm is more elitist  and requiere more computations.
# the main idea is that here paretns ( mu) and child(lambda) compite between them
# The paremeters are initial population, toolbox, mu, lambda, cxpb ( probability of crossover)
# mutpb (probability of mutation), ngen (number of generations), stats ( registry of functions), halloffame


def main():
    random.seed(101)
    CXPB, MUTPB, NGEN = 0.95, 0.05, 300
    pop = toolbox.population()
    MU, LAMBDA = len(pop), len(pop)
    hof = tools.HallOfFame(1)
    stats = tools.Statistics(lambda ind: ind.fitness.values)
    stats.register("avg", np.mean)
    stats.register("std", np.std)
    stats.register("min", np.min)
    stats.register("max", np.max)
    logbook = tools.Logbook()
    pop, logbook = algorithms.eaMuPlusLambda(pop,toolbox, MU, LAMBDA, CXPB, MUTPB, NGEN,
                                            stats = stats, halloffame=hof)
    return pop, logbook

if __name__ == "__main__":
    best, log  = main()
    print("\n",'--'*50)
    print(f"The fitness {best[0].fitness.values} and he better path is : \n",)
    print(capitals[best[0]])
    print("--"*50)

/usr/local/lib/python3.11/dist-packages/deap/creator.py:185: RuntimeWarning: A class named 'FitnessMin' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "
/usr/local/lib/python3.11/dist-packages/deap/creator.py:185: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "


gen	nevals	avg    	std    	min    	max    
0  	100   	147.391	12.5071	117.315	175.442
1  	100   	127.279	7.07992	109.605	141.948
2  	100   	116.498	5.15089	109.605	128.869
3  	100   	109.718	1.61302	106.452	117.315
4  	100   	106.644	2.75023	101.161	109.605
5  	100   	101.088	2.99526	92.981 	107.491
6  	100   	97.1304	3.28016	92.981 	101.161
7  	100   	92.8717	0.781998	91.1787	97.763 
8  	100   	92.037 	0.798802	91.1787	92.981 
9  	100   	91.2012	0.127469	91.1787	91.926 
10 	100   	91.1787	1.42109e-14	91.1787	91.1787
11 	100   	91.1567	0.095914   	90.7387	91.1787
12 	100   	90.9499	0.219866   	90.7387	91.1787
13 	100   	90.502 	1.11805    	85.0275	90.7387
14 	100   	89.4355	2.27806    	85.0275	90.7387
15 	100   	85.3444	1.47944    	84.4241	90.5753
16 	100   	84.5199	0.258943   	83.9075	85.0275
17 	100   	84.0917	0.63975    	80.7499	84.4241
18 	100   	83.1571	1.31974    	80.7499	84.4241
19 	100   	80.7881	0.731225   	79.6626	83.9075
20 	100   	80.1402	0.553359   	78.7557	80.7499
21 	100

In [18]:
import folium
map = folium.Map(location=[4.570868, -74.297333], zoom_start=6)
points = [cities_dict[city] for city in list(capitals[best[0]]) + [capitals[best[0][0]]]]
folium.PolyLine(points).add_to(map)
map